In [6]:
import numpy as np
import pandas as pd
import json
import os

import plotly.express as px
from tqdm import tqdm

from itertools import chain

In [7]:
#os.chdir("../eval-MCG-COLING-2025") # If needed set wd to root of project
#!pwd

# General functions

Formatiting functions

In [12]:
def create_win_col(battles,col="human_wins",name="Winner"):
    """Creates a column with the actual name of the winner model. tie if no winner"""

    battles[col]  = battles.apply(lambda x: x["model_a"] if x[name] == "A" else "tie", axis=1)
    battles[col]  = battles.apply(lambda x: x["model_b"] if x[name] == "B" else "tie", axis=1)
    return(battles)

In [13]:
def create_rank(battles,wins="human_wins"):
    """Score each model's performance and return a ranking"""
    
    battles["tie"]=battles.apply(lambda row: [row["model_a"],row["model_b"]] if row[wins]=="tie" else np.nan, axis=1)

    winner_cn=battles[wins].value_counts().to_dict()
    try:    
        del winner_cn["tie"]
    except:
        print (i," no tie")

    #Ties
    list_of_lists =battles.loc[battles["tie"].notna()]["tie"].tolist()
    flattened_list = list(chain(*list_of_lists))
    tie_count=pd.Series(flattened_list).value_counts().to_dict()
    tie_score={key:value/2 for key,value in tie_count.items()}

    total= {k: winner_cn.get(k, 0) + tie_score.get(k, 0) for k in set(winner_cn) | set(tie_score)}
    total = dict(sorted(total.items(), key=lambda item: item[1], reverse=True))
    return(total)

def plot_rank(rank):
    fig = px.bar(x=list(rank.keys()), y=list(rank.values()),
                title="Counts of Battle Outcomes", text_auto=True, height=400)
    fig.update_layout(xaxis_title="Battle Outcome", yaxis_title="Count",
                    showlegend=False)
    return fig

## DataFrame with wins

In [55]:
judgements=pd.read_csv("./judgements/7_ilv_results.csv")
display(judgements)

,question_id,judge_model,model_a,model_b,wins
0,0,7b-JudgeLM,gold_truth,system_1,system_1
1,1,7b-JudgeLM,gold_truth,system_1,system_1
2,2,7b-JudgeLM,gold_truth,system_1,system_1
3,3,7b-JudgeLM,gold_truth,system_1,system_1
4,4,7b-JudgeLM,gold_truth,system_1,system_1
5,5,7b-JudgeLM,gold_truth,system_1,system_1
6,6,7b-JudgeLM,gold_truth,system_1,system_1
7,7,7b-JudgeLM,gold_truth,system_1,gold_truth
8,8,7b-JudgeLM,gold_truth,system_1,system_1
9,9,7b-JudgeLM,gold_truth,system_1,gold_truth


In [17]:
!ls

automatic_evaluation  __init__.py   judgements	       scripts
bash		      JudgeLM-main  Rank_models.ipynb


## Traditional metrics

To rank by the repetition rate change, set the ascending parameter to True. For all other metrics, please retain their current settings. Note that generation length will not be used for ranking; it serves only as an informative feature.

In [50]:
pd.read_csv("./automatic_evaluation/traditional_metrics2.csv").sort_values(by='bertscore', ascending=False)

,model,rougeL,bleu,bertscore,gen_len,novelty
0,system_1,0.1483,0.0205,0.6788,28.916667,0.8065
1,system_2,0.1483,0.0205,0.6788,28.916667,0.8065


## LLM Evaluator

In [56]:
judge_rank=create_rank(judgements,"wins")

In [57]:
plot_rank(judge_rank) #higher is better